In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# Step 1: Load the text from a .txt file
file_path = 'wood_corpus.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
# Step 2: Load pre-trained RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaModel.from_pretrained('roberta-large')

In [ ]:
# Step 3: Tokenize the text
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

#see tokens:
input_ids = inputs['input_ids'][0]
tokens = tokenizer.convert_ids_to_tokens(input_ids)

print("Tokens:", tokens)

In [ ]:
# Step 4: Extract embeddings
# Get the tokenized input IDs and attention mask
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Run the inputs through the model
with torch.no_grad():
    
    outputs = model(input_ids, attention_mask=attention_mask)
    # Get the last hidden states (embeddings for each token)
    embeddings = outputs.last_hidden_state

In [ ]:
# List of words to extract embeddings for
words = ["heartwood", "sapwood"]

# Tokenize the individual words
words_ids = {word: tokenizer.encode(word, add_special_tokens=False) for word in words}

# Initialize a dictionary to store embeddings
embeddings_dict = {}

# Loop through each word in the list
for word, word_id in words_ids.items():
    
    # Find the indices of this word in the tokenized input
    indices = [i for i, id_ in enumerate(input_ids[0]) if id_ in word_id]
    
    # Extract embeddings for the specific word
    embedding = embeddings[0, indices, :].mean(dim=0) if indices else None
    
    # Store the embedding in the dictionary
    embeddings_dict[word] = embedding

# Output the embeddings
for word, embedding in embeddings_dict.items():
    
    print(f"Embedding for {word}: {embedding}")
    
heartwood_embedding = embeddings_dict["heartwood"]
sapwood_embedding = embeddings_dict["sapwood"]

In [ ]:
print(heartwood_embedding.shape, sapwood_embedding.shape)

similarity = cosine_similarity(heartwood_embedding.unsqueeze(0), sapwood_embedding.unsqueeze(0))
print("Cosine Similarity:", similarity)

In [ ]:
# Save the embeddings to a .npy file
np.save("Wood_Embeddings.npy", embeddings_dict)

In [ ]:
#load the embeddings back to verify
loaded_embeddings = np.load("Wood_Embeddings.npy", allow_pickle=True).item()

#print(loaded_embeddings.shape)
print(loaded_embeddings)

In [ ]:
'''
# Assume embedding_1 is from my example and embedding_2 is from your example
diff = heartwood_embedding - embedding_word_1

# Check if all elements are zero (exact match)
is_exact = torch.equal(heartwood_embedding, embedding_word_1)

# Alternatively, check for approximate equality (handles floating-point precision issues)
is_approx_equal = torch.allclose(heartwood_embedding, embedding_word_1, atol=1e-6)

print("Are the embeddings exactly the same?", is_exact)
print("Are the embeddings approximately the same?", is_approx_equal)
'''